In [1]:
import arcpy, os, timeit
from arcpy import env

In [2]:
MSAGrange = os.path.join(r'G:\projects\Address_Points\9-1-1_Net', 'MSAG_range_match.shp')

In [3]:
AddressPoints = os.path.join(r'G:\projects\Address_Points\9-1-1_Net', 'AddressPoints.shp')

In [4]:
arcpy.MakeFeatureLayer_management(AddressPoints,"AddressPoints")
fieldList = arcpy.ListFields("AddressPoints")
field_names = [f.name for f in fieldList]
newfields = ["DIFF", "MATCH"]
for newfield in newfields:
    if newfield in field_names:
        pass
    else:
        if newfield == "DIFF":
            arcpy.AddField_management("AddressPoints", newfield, "SHORT", "", "", 10)
        else:
            arcpy.AddField_management("AddressPoints", newfield, "TEXT", "", "", 1000)     

In [5]:
arcpy.MakeFeatureLayer_management(MSAGrange,"ExactMatch")

<Result 'ExactMatch'>

In [6]:
arcpy.SelectLayerByAttribute_management("ExactMatch", "NEW_SELECTION", "DIFF = 1")

id,value
0,a Layer object
1,7331


In [7]:
arcpy.SelectLayerByLocation_management("AddressPoints", "INTERSECT", "ExactMatch", "", "NEW_SELECTION")

id,value
0,a Layer object
1,AddressPoints
2,173026


In [8]:
with arcpy.da.UpdateCursor("AddressPoints", ['DIFF', 'MATCH']) as cursor:
    for row in cursor:
        row[0] = 1
        row[1] = "Exactly matched"
        cursor.updateRow(row)
del row
del cursor

In [9]:
arcpy.SelectLayerByAttribute_management("AddressPoints", "CLEAR_SELECTION")

id,value
0,a Layer object
1,-1


In [10]:
MSAG_mismatch = os.path.join(r'G:\projects\Address_Points\9-1-1_Net', 'MSAG_mismatch.shp')

In [11]:
MisMatch = arcpy.MakeFeatureLayer_management(MSAG_mismatch,"MSAG_mismatch")

In [12]:
start = timeit.default_timer()
with arcpy.da.UpdateCursor(MisMatch, ['FID', "DIFF", 'MATCH']) as cursor1:
    for row1 in cursor1:
        selected = arcpy.SelectLayerByAttribute_management(MisMatch, "NEW_SELECTION", "FID = {0}".format(row1[0]))
        arcpy.SelectLayerByLocation_management("AddressPoints", "INTERSECT", selected, "", "NEW_SELECTION")   
        with arcpy.da.UpdateCursor("AddressPoints", ["DIFF", "MATCH"]) as cursor2:
            for row2 in cursor2:
                row2[0] = row1[1]
                row2[1] = row1[2]
                cursor2.updateRow(row2)
del row1
del cursor1
del row2
del cursor2
stop = timeit.default_timer()
print(stop - start)

133.7054007
